# Day 3 - Conversational AI - aka Chatbot!

In [19]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [12]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
api_key = os.getenv("GEMINI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key not set
Anthropic API Key not set
Google API Key not set


In [53]:
# Initialize

# openai = OpenAI()
# MODEL = 'gpt-4o-mini'
MODEL = "gemini-2.0-flash"
model = ChatGoogleGenerativeAI(model=MODEL, api_key=api_key)

In [54]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [55]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

# Student Octavio O. has pointed out that this isn't quite as straightforward for Claude -
# see the excellent contribution in community-contributions "Gradio_issue_with_Claude" that handles Claude.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [56]:
conversation_history = []  
def stream_chat(user_input, model_name):
    global conversation_history

    # Convert previous history to LangChain messages
    lc_history = []
    for h in conversation_history:
        role = h["role"]
        content = h["content"]
        if role == "user":
            lc_history.append(HumanMessage(content=content))
        elif role == "assistant":
            lc_history.append(AIMessage(content=content))
        else:
            lc_history.append(SystemMessage(content=content))

    # Full messages: system + history + current user
    messages = [SystemMessage(content=system_message)] + lc_history + [HumanMessage(content=user_input)]
    print(f"{messages}\n\n\n")

    # Stream the response from the model
    response_text = ""
    for chunk in model.stream(messages):
        if chunk.content:
            response_text += chunk.content
            yield response_text

    # Save to conversation history for next prompt
    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": response_text})
    print(f"{conversation_history}\n\n\n")


## And then enter Gradio's magic!

In [57]:
gr.ChatInterface(fn=stream_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7907
* To create a public link, set `share=True` in `launch()`.


In [58]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [59]:
conversation_history = []  
def stream_chat(user_input,model_name):
    global conversation_history

    # Convert previous history to LangChain messages
    lc_history = []
    for h in conversation_history:
        role = h["role"]
        content = h["content"]
        if role == "user":
            lc_history.append(HumanMessage(content=content))
        elif role == "assistant":
            lc_history.append(AIMessage(content=content))
        else:
            lc_history.append(SystemMessage(content=content))

    # Full messages: system + history + current user
    messages = [SystemMessage(content=system_message)] + lc_history + [HumanMessage(content=user_input)]
    print(f"{messages}\n\n\n")

    # Stream the response from the model
    response_text = ""
    for chunk in model.stream(messages):
        if chunk.content:
            response_text += chunk.content
            yield response_text

    # Save to conversation history for next prompt
    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": response_text})
    print(f"{conversation_history}\n\n\n")


In [60]:
gr.ChatInterface(fn=stream_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.


[SystemMessage(content="You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.", additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]



[{'role': 'user', 'content': 'hi'}, {'role': 'assistant', 'content': 'Hi there! Welcome to our store. Are you looking for anything in particular today? We have a great sales event going on right now, with most items 50% off! And all our hats are an amazing 60% off!\n'}]





In [61]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [62]:
print(type(model))

<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>


In [64]:
gr.ChatInterface(fn=stream_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7910
* To create a public link, set `share=True` in `launch()`.


[SystemMessage(content="You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!", additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! Welcome to our store. Are you looking for anything in particular today? We have a great sales event going on right now, with most items 50% off! And all our hats are an amazing 60% off!\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_

In [ ]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [68]:
print(type(model))

<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>


In [71]:
conversation_history = []

def stream_chat(user_input,m):
    global conversation_history

    # Update system message if 'belt' is mentioned
    relevant_system_message = system_message
    if 'belt' in user_input.lower():  # lowercase to make it case-insensitive
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."

    # Convert previous history to LangChain messages
    lc_history = []
    for h in conversation_history:
        role = h["role"]
        content = h["content"]
        if role == "user":
            lc_history.append(HumanMessage(content=content))
        elif role == "assistant":
            lc_history.append(AIMessage(content=content))
        else:
            lc_history.append(SystemMessage(content=content))

    # Full messages: system + history + current user
    messages = [SystemMessage(content=relevant_system_message)] + lc_history + [HumanMessage(content=user_input)]
    print(f"Messages:\n{messages}\n\n")

    # Stream the response from the model
    response_text = ""
    for chunk in model.stream(messages):
        if chunk.content:
            response_text += chunk.content
            yield response_text

    # Save to conversation history for next prompt
    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": response_text})
    print(f"Conversation history:\n{conversation_history}\n\n")


In [72]:
gr.ChatInterface(fn=stream_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7914
* To create a public link, set `share=True` in `launch()`.


Messages:
[SystemMessage(content="You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!", additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]


Conversation history:
[{'role': 'user', 'content': 'hi'}, {'role': 'assistant', 'content': "Hello! Welcome to our store. What are you looking for today? I'd be happy to help you find something. We have a great sales event happening right now, with many items at 50% off! And don't forget to

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>